In [52]:
import glob
import pandas as pd 
import xml.etree.ElementTree as ET 
from datetime import datetime

## TASK 1: EXTRACTION

### Lưu ý quan trọng

- **Import ElementTree**:  
  Lưu ý rằng bạn chỉ import hàm `ElementTree` từ thư viện `xml.etree` vì bạn cần hàm này để phân tích dữ liệu từ tệp có định dạng XML.

- **Đường dẫn tệp (File Paths)**:  
  Bạn cũng cần hai đường dẫn tệp được khai báo toàn cục (globally) trong mã nguồn để sử dụng cho tất cả các hàm:
  1. **`transformed_data.csv`**: Để lưu trữ dữ liệu đầu ra cuối cùng có thể tải lên cơ sở dữ liệu.
  2. **`log_file.txt`**: Để lưu trữ tất cả các bản ghi (logs).


In [53]:
log_file = "log_file.txt"
target_file = "transformed_data.csv"

### Phát triển các hàm trích xuất

- **Mục đích**:  
  Tiếp theo, bạn sẽ phát triển các hàm để trích xuất dữ liệu từ các định dạng tệp khác nhau. Vì mỗi định dạng tệp sẽ yêu cầu một hàm riêng, bạn cần viết một hàm cho từng loại: `.csv`, `.json` và `.xml`.

- **Tên hàm**:  
  Bạn có thể đặt tên cho ba hàm này như sau:
  1. **`extract_from_csv()`**
  2. **`extract_from_json()`**
  3. **`extract_from_xml()`**

- **Tham số (Parameters)**:  
  Mỗi hàm sẽ nhận một tham số đầu vào là tệp dữ liệu, được đặt tên là `file_to_process`.

- **Trích xuất từ CSV**:  
  Để trích xuất dữ liệu từ tệp CSV, bạn có thể định nghĩa hàm `extract_from_csv()` như sau bằng cách sử dụng hàm `read_csv` của thư viện pandas.


In [54]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe 

### Trích xuất dữ liệu từ tệp JSON

- **Trích xuất từ JSON**:  
  Để trích xuất dữ liệu từ tệp JSON, bạn có thể định nghĩa hàm `extract_from_json()` bằng cách sử dụng hàm `read_json` của thư viện pandas.

- **Tham số bổ sung**:  
  Hàm yêu cầu một tham số bổ sung, `lines=True`, để cho phép đọc tệp như một đối tượng JSON theo từng dòng.


In [55]:
def extract_from_json(file_to_process): 
    dataframe = pd.read_json(file_to_process, lines=True) 
    return dataframe 

### Trích xuất dữ liệu từ tệp XML

- **Trích xuất từ XML**:  
  Để trích xuất dữ liệu từ tệp XML, bạn cần phân tích dữ liệu trước bằng cách sử dụng hàm `ElementTree`. Sau khi dữ liệu được phân tích, bạn có thể trích xuất thông tin cần thiết và thêm chúng vào một DataFrame của pandas.

- **Lưu ý**:  
  Bạn cần biết các tiêu đề (headers) của dữ liệu được trích xuất để viết hàm này. Trong ví dụ này, các tiêu đề là `"name"`, `"height"`, và `"weight"` đại diện cho thông tin của các cá nhân khác nhau.

- **Ví dụ**:  
  Quy trình cơ bản:
  1. Phân tích tệp XML bằng `ElementTree`.
  2. Lặp qua dữ liệu đã phân tích để trích xuất các trường cần thiết.
  3. Thêm dữ liệu vào DataFrame của pandas.

In [56]:
def extract_from_xml(file_to_process): 
    dataframe = pd.DataFrame(columns=["name", "height", "weight"]) 
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    for person in root: 
        name = person.find("name").text 
        height = float(person.find("height").text) 
        weight = float(person.find("weight").text) 
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True) 
    return dataframe 

Bây giờ, bạn cần một hàm để xác định hàm nào sẽ được gọi dựa trên loại tệp của tệp dữ liệu. Để gọi hàm phù hợp, hãy viết một hàm `extract`, sử dụng thư viện `glob` để nhận diện loại tệp. Có thể thực hiện điều này như sau:


Tham số ignore_index=True đảm bảo các chỉ số (index) được tự động sắp xếp lại.

In [57]:

def extract(): 
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data 
     
    # process all csv files 
    for csvfile in glob.glob("*.csv"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True) 
         
    # process all json files 
    for jsonfile in glob.glob("*.json"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True) 
     
    # process all xml files 
    for xmlfile in glob.glob("*.xml"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True) 
         
    return extracted_data 

## TASK 2: TRANSFORMATION

### Hàm Transform: Chuyển đổi đơn vị

**Vấn đề**
- **Chiều cao** trong dữ liệu trích xuất được đo bằng **inch**, và **cân nặng** được đo bằng **pound**.
- Yêu cầu ứng dụng:
  - Chiều cao phải được chuyển đổi sang **mét**.
  - Cân nặng phải được chuyển đổi sang **kilogram**, làm tròn đến **hai chữ số thập phân**.

**Giải pháp**
- Viết một hàm có tên là **`transform()`** để thực hiện chuyển đổi đơn vị cho hai tham số này.
- Hàm sẽ:
  1. Nhận DataFrame đã trích xuất làm đầu vào.
  2. Áp dụng chuyển đổi đơn vị cho danh sách chiều cao và cân nặng.

**Ghi chú**
- DataFrame đầu vào có cấu trúc dạng dictionary với ba khóa:
  1. `"name"`
  2. `"height"`
  3. `"weight"`

- Mỗi khóa chứa một danh sách các giá trị, cho phép xử lý toàn bộ danh sách trong một lần.


In [58]:
def transform(data):
    data['height'] = round(data.height * 0.0254, 2)
    data['weight'] = round(data.weight * 0.453592, 2)
    return data


## TASK 3: LOAD

### Hàm Load: Lưu dữ liệu đã chuyển đổi vào tệp CSV

**Yêu cầu**
- Dữ liệu đã chuyển đổi cần được lưu vào tệp **CSV** để sử dụng cho việc tải lên cơ sở dữ liệu.

**Giải pháp**
- Viết một hàm có tên là **`load_data()`** để thực hiện việc lưu dữ liệu.
- Hàm:
  1. Nhận dữ liệu đã chuyển đổi dưới dạng **DataFrame**.
  2. Nhận đường dẫn **target_file** nơi dữ liệu sẽ được lưu.
  3. Sử dụng thuộc tính `to_csv` của DataFrame để ghi dữ liệu vào tệp được chỉ định.

In [59]:
def load_data(target_file, transformed_data):
    transformed_data.to_csv(target_file)

### Hàm Logging: Ghi nhận tiến trình của các thao tác

**Yêu cầu**
- Để theo dõi tiến trình, bạn cần ghi nhận một **thông điệp log** kèm theo **dấu thời gian** vào tệp log (`log_file`).

**Giải pháp**
- Triển khai một hàm có tên là **`log_progress()`**, với các bước:
  1. Nhận thông điệp log làm tham số đầu vào.
  2. Lấy **ngày giờ hiện tại** bằng cách sử dụng hàm `datetime` từ thư viện `datetime`.
  3. Chuyển đổi dấu thời gian thành chuỗi bằng thuộc tính `strftime` với định dạng ngày-giờ được xác định.
  4. Ghi thông điệp kèm dấu thời gian vào tệp log.

In [60]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

## TEST ETL OPERATIONS

In [63]:
log_process("ETL Job Started")

log_process("Extract phase Started")
extracted_data = extract()

log_process("Extract phase Ended")

log_process("Transform phase Started")
transformed_data = transform(extracted_data)
log_process("Transform phase Ended")

log_process("Load phase Started")
load_data(target_file, transformed_data)

# Log the completion of the Loading process 
log_progress("Load phase Ended") 
 
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 

C:\Users\baoho\AppData\Local\Temp\ipykernel_20020\2154094443.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)
C:\Users\baoho\AppData\Local\Temp\ipykernel_20020\2394561360.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True)
C:\Users\baoho\AppData\Local\Temp\ipykernel_20020\